Decision trees are a non-parametric model used for both regression and
classification tasks

-This notebook demonstrates how to use decision trees for classification

In [1]:
#Example in Python

import numpy as np
from collections import Counter

In [2]:
def entropy(s):
    counts = np.bincount(s)
    percentages = counts / len(s)

    entropy = 0
    for pct in percentages:
        if pct > 0:
            entropy += pct * np.log2(pct)
    return -entropy

In [3]:
s = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
print(f'Entropy: {np.round(entropy(s), 5)}')

Entropy: 0.88129


In [4]:
#The higher the information gain, the better the decision split is
#Example in Python

def information_gain(parent, left_child, right_child):
    num_left = len(left_child) / len(parent)
    num_right = len(right_child) / len(parent)

    gain = entropy(parent) - (num_left * entropy(left_child) + num_right * entropy(right_child))
    return gain

In [5]:
parent = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
left_child = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
right_child = [0, 0, 0, 0, 1, 1, 1, 1]

print(f'Information gain: {np.round(information_gain(parent, left_child, right_child), 5)}')

Information gain: 0.18094


***Recursion Crash Course***
A lot of decision trees implementation boils down to recursion

Let's take a look at the simplest example - a recursive function that returns a
factorial of an integer

In [6]:
def factorial(x):
    # Exit condition
    if x == 1:
        return 1
    return x * factorial(x - 1)

print(f'Factorial of 5 is {factorial(5)}')

Factorial of 5 is 120


***Implementation*** We'll need two classes


Node - implements a single node of a decision tree


DecisionTree - implements the algorithm

In [7]:
class Node:
    '''
    Helper class which implements a single tree node.
    '''
    def __init__(self, feature=None, threshold=None, data_left=None, data_right=None, gain=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.data_left = data_left
        self.data_right = data_right
        self.gain = gain
        self.value = value

In [8]:
class DecisionTree:
    '''
    Class which implements a decision tree classifier algorithm.
    '''
    def __init__(self, min_samples_split=2, max_depth=5):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None

    @staticmethod
    def _entropy(s):
        '''
        Helper function, calculates entropy from an array of integer values.

        :param s: list
        :return: float, entropy value
        '''
        # Convert to integers to avoid runtime errors
        counts = np.bincount(np.array(s, dtype=np.int64))
        # Probabilities of each class label
        percentages = counts / len(s)

        # Caclulate entropy
        entropy = 0
        for pct in percentages:
            if pct > 0:
                entropy += pct * np.log2(pct)
        return -entropy

    def _information_gain(self, parent, left_child, right_child):
        '''
        Helper function, calculates information gain from a parent and two child nodes.

        :param parent: list, the parent node
        :param left_child: list, left child of a parent
        :param right_child: list, right child of a parent
        :return: float, information gain
        '''
        num_left = len(left_child) / len(parent)
        num_right = len(right_child) / len(parent)

        # One-liner which implements the previously discussed formula
        return self._entropy(parent) - (num_left * self._entropy(left_child) + num_right *
self._entropy(right_child))
        
    def _best_split(self, X, y):
        '''
        Helper function, calculates the best split for given features and target

        :param X: np.array, features
        :param y: np.array or list, target
        :return: dict
        '''
        best_split = {}
        best_info_gain = -1
        n_rows, n_cols = X.shape

        # For every dataset feature
        for f_idx in range(n_cols):
            X_curr = X[:, f_idx]
            # For every unique value of that feature
            for threshold in np.unique(X_curr):
                # Construct a dataset and split it to the left and right parts
                # Left part includes records lower or equal to the threshold
                # Right part includes records higher than the threshold
                df = np.concatenate((X, y.reshape(1, -1).T), axis=1)
                df_left = np.array([row for row in df if row[f_idx] <= threshold])
                df_right = np.array([row for row in df if row[f_idx] > threshold])

                # Do the calculation only if there's data in both subsets
                if len(df_left) > 0 and len(df_right) > 0:
                    # Obtain the value of the target variable for subsets
                    y = df[:, -1]
                    y_left = df_left[:, -1]
                    y_right = df_right[:, -1]

                    # Caclulate the information gain and save the split parameters
                    # if the current split if better then the previous best
                    gain = self._information_gain(y, y_left, y_right)
                    if gain > best_info_gain:
                        best_split = {
                            'feature_index': f_idx,
                            'threshold': threshold,
                            'df_left': df_left,
                            'df_right': df_right,
                            'gain': gain
                        }
                        best_info_gain = gain
        return best_split
def _build(self, X, y, depth=0):
    '''
    Helper recursive function, used to build a decision tree from the input data.

    :param X: np.array, features
    :param y: np.array or list, target
    :param depth: current depth of a tree, used as a stopping criteria
    :return: Node
    '''
    n_rows, n_cols = X.shape

    # Check to see if a node should be leaf node
    if n_rows >= self.min_samples_split and depth <= self.max_depth:
        # Get the best split
        best = self._best_split(X, y)
        # If the split isn't pure
        if best['gain'] > 0:
            # Build a tree on the left
            left = self._build(
                X=best['df_left'][:, :-1],
                y=best['df_left'][:, -1],
                depth=depth + 1
            )
            right = self._build(
                X=best['df_right'][:, :-1],
                y=best['df_right'][:, -1],
                depth=depth + 1
            )
            return Node(
                feature=best['feature_index'],
                threshold=best['threshold'],
                data_left=left,
                data_right=right,
                gain=best['gain']
            )
    # Leaf node - value is the most common target value
    return Node(
        value=Counter(y).most_common(1)[0][0]
    )
def fit(self, X, y):
    '''
    Function used to train a decision tree classifier model.

    :param X: np.array, features
    :param y: np.array or list, target
    :return: None
    '''
    # Call a recursive function to build the tree
    self.root = self._build(X, y)

def _predict(self, x, tree):
    '''
    Helper recursive function, used to predict a single instance (tree traversal).

    :param x: single observation
    :param tree: built tree
    :return: float, predicted class
    '''
    # Leaf node
    if tree.value != None:
        return tree.value
    feature_value = x[tree.feature]

    # Go to the left
    if feature_value <= tree.threshold:
        return self._predict(x=x, tree=tree.data_left)

    # Go to the right
    if feature_value > tree.threshold:
        return self._predict(x=x, tree=tree.data_right)

def predict(self, X):
    '''
    Function used to classify new instances.

    :param X: np.array, features
    :return: np.array, predicted classes
    '''
    # Call the _predict() function for every observation
    return [self._predict(x, self.root) for x in X]

***We'll use the Iris dataset from Scikit-Learn for Testing***

In [9]:
from sklearn.datasets import load_iris

iris = load_iris()

X = iris['data']
y = iris['target']

In [10]:
#The below code applies train/test split to the dataset:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
#initialize and train the model for prediction

model = DecisionTree()
model.fit(X_train, y_train)
preds = model.predict(X_test)

AttributeError: 'DecisionTree' object has no attribute 'fit'

In [ ]:
np.array(preds, dtype=np.int64)

In [ ]:
y_test

In [ ]:
#Let's calculate the accuracy to confirm

from sklearn.metrics import accuracy_score
accuracy_score(y_test, preds)

***Comparison with Scikit-Learn***


In [ ]:
from sklearn.tree import DecisionTreeClassifier

sk_model = DecisionTreeClassifier()
sk_model.fit(X_train, y_train)
sk_preds = sk_model.predict(X_test)

In [ ]:
accuracy_score(y_test, sk_preds)

#Both perform the same, at least accuracy-wise